In [1]:
# -*- coding: utf-8 -*-
import sys
import gym
print sys.getdefaultencoding()

ascii


In [2]:
import keras

Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\DELL\Anaconda2\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
import random
import math
import numpy as np
from collections import deque

In [4]:
#Setup OpenAi gym environment
env = gym.make('CartPole-v0')
for i_episode in range(20):
    observation = env.reset()
    env.render()
    for t in range(1000):
        print(observation)
        action = env.action_space.sample()
        observation,reward,done,info = env.step(action)
        
        if done:
            print("Episode finished after {} timestamps".format(t+1))
            break

[-0.0382476   0.03741649 -0.0482892   0.03696975]
[-0.03749927 -0.15698092 -0.04754981  0.31403445]
[-0.04063889  0.038785   -0.04126912  0.00674309]
[-0.03986319  0.23447377 -0.04113426 -0.2986698 ]
[-0.03517371  0.03996156 -0.04710765 -0.01923836]
[-0.03437448  0.23572633 -0.04749242 -0.32640428]
[-0.02965996  0.43149115 -0.0540205  -0.63367793]
[-0.02103013  0.6273234  -0.06669406 -0.9428721 ]
[-0.00848367  0.82327752 -0.0855515  -1.25574333]
[ 0.00798188  1.01938421 -0.11066637 -1.573949  ]
[ 0.02836957  1.21563827 -0.14214535 -1.89899883]
[ 0.05268233  1.41198469 -0.18012533 -2.23219818]
Episode finished after 12 timestamps
[-0.04704624 -0.03719734  0.00961494  0.03377508]
[-0.04779018 -0.23245584  0.01029044  0.32947604]
[-0.0524393  -0.03748188  0.01687997  0.04005592]
[-0.05318894 -0.23284177  0.01768108  0.33801649]
[-0.05784577 -0.4282108   0.02444141  0.6362222 ]
[-0.06640999 -0.62366495  0.03716586  0.93650091]
[-0.07888329 -0.81926787  0.05589588  1.24062692]
[-0.09526864 

[-0.12127334 -0.83213234  0.15452849  1.34730179]
[-0.13791599 -0.6392532   0.18147453  1.10668132]
[-0.15070105 -0.83623586  0.20360815  1.45036437]
Episode finished after 13 timestamps
[0.00125374 0.00636727 0.04963237 0.00677122]
[ 0.00138108  0.20074358  0.04976779 -0.26984828]
[ 0.00539596  0.39512131  0.04437083 -0.54642817]
[ 0.01329838  0.19940494  0.03344226 -0.24010182]
[0.01728648 0.00382162 0.02864023 0.06293932]
[ 0.01736291 -0.19169901  0.02989901  0.36451906]
[0.01352893 0.00298555 0.0371894  0.08141175]
[ 0.01358864  0.1975552   0.03881763 -0.19930977]
[0.01753975 0.00190017 0.03483144 0.10536121]
[ 0.01757775 -0.19370318  0.03693866  0.40882664]
[ 0.01370369 -0.38932886  0.04511519  0.71292248]
[ 0.00591711 -0.58504546  0.05937364  1.01945803]
[-0.0057838  -0.78090628  0.0797628   1.33017718]
[-0.02140192 -0.58687599  0.10636635  1.06348249]
[-0.03313944 -0.78323283  0.127636    1.38756594]
[-0.0488041  -0.58991165  0.15538732  1.13736617]
[-0.06060233 -0.7866862   0.1

In [5]:
print(env.action_space)
print(env.observation_space)

Discrete(2)
Box(4L,)


In [6]:
print(env.observation_space.high)
print(env.observation_space.low)

[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]


In [7]:
#Training parameters
n_episodes = 1000
n_win_ticks = 195
max_env_steps = None

gama = 1.0 #Discount factor
epsilon = 1.0 #Exploration
epsilon_min = 0.01
epsilon_decay = 0.995
alpha = 0.01  #the leanring rate
alpha_decay = 0.01

batch_size = 64
monitor = False
quiet = False

#Environment parameters
memory = deque(maxlen=100000)
env = gym.make('CartPole-v0')
if max_env_steps is not None : env.max_episode_steps = max_env_steps

In [8]:
#Building the neural network

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

#Model Parameters
model = Sequential()
model.add(Dense(24,input_dim=4,activation='relu'))
model.add(Dense(48,activation='relu'))
model.add(Dense(2,activation='relu'))
model.compile(loss='mse' ,optimizer=Adam(lr=alpha,decay=alpha_decay))


In [12]:
#Defining necessary function

def remember(state,action,reward,next_state,done):
    memory.append(state,action,reward,next_state,done)

def choose_action(state,epsilon):
    return env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(model.predict(state))

def get_epsilon(t):
    return max(epsilon_min,min(epsilon,1.0 - math.log10(t+1)*epsilon_decay))

def preprocess_state(state):
    return np.reshape(state,[1,4])

def replay(batch_size,epsilon):
    x_batch,y_batch=[],[]
    minibatch = random.sample(memory,min(len(memory),batch_size))
    
    for state,action,reward,next_state,done in minibatch:
        y_target=model.predict(state)
        y_target[0][action]=reward if done else reward + gama*np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])
    
    model.fit(np.array(x_batch),np.array(y_batch),batch_size=len(x_batch),verbose=0)
    
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

In [13]:
#Define run function
def run():
    scores = deque(maxlen=100)
    
    for e in range(n_episode):
        state = preprocess_state(env.reset())
        done = False
        i=0
        while not done:
            action = choose_action(state,get_epsilon(e))
            next_state,reward,done , _ = env.step(action)
            env.render()
            next_state = preprocess_state(next_state)
            remember(state,action,reward,next_state,done)
            state=next_state
            i=i+1
        scores.append(i)
        mean_score = np.mean(scores)
        
        if(mean_score >= n_win_ticks and e >100):
            if not quiet : print ("Ran {} episodes. Solved after {} trails".format(e,e-100))
            return e -100
        
        if e%100==0 and not quiet:
            print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks '.format(e,mean_score))
            
        replay(batch_size,epsilon)
        
    if not quiet: print('Did not solve after {} episodes'.format(e))
    return e

In [ ]:
# Training the network
# -*- coding: utf-8 -*-
import sys
import gym
import keras
import random
import math
import numpy as np
from collections import deque

#Training parameters
n_episodes = 1000
n_win_ticks = 195
max_env_steps = None

gama = 1.0 #Discount factor
epsilon = 1.0 #Exploration
epsilon_min = 0.01
epsilon_decay = 0.995
alpha = 0.01  #the leanring rate
alpha_decay = 0.01

batch_size = 64
monitor = False
quiet = False

#Environment parameters
memory = deque(maxlen=100000)
env = gym.make('CartPole-v0')
if max_env_steps is not None : env.max_episode_steps = max_env_steps
    
    
    
#Building the neural network

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

#Model Parameters
model = Sequential()
model.add(Dense(24,input_dim=4,activation='relu'))
model.add(Dense(48,activation='relu'))
model.add(Dense(2,activation='relu'))
model.compile(loss='mse' ,optimizer=Adam(lr=alpha,decay=alpha_decay))

#Defining necessary function

def remember(state,action,reward,next_state,done):
    memory.append((state,action,reward,next_state,done))

def choose_action(state,epsilon):
    return env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(model.predict(state))

def get_epsilon(t):
    return max(epsilon_min,min(epsilon,1.0 - math.log10(t+1)*epsilon_decay))

def preprocess_state(state):
    return np.reshape(state,[1,4])

def replay(batch_size,epsilon):
    x_batch,y_batch=[],[]
    minibatch = random.sample(memory,min(len(memory),batch_size))
    
    for state,action,reward,next_state,done in minibatch:
        y_target=model.predict(state)
        y_target[0][action]=reward if done else reward + gama*np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])
    
    model.fit(np.array(x_batch),np.array(y_batch),batch_size=len(x_batch),verbose=0)
    
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay
        
        
#Define run function
def run():
    scores = deque(maxlen=100)
    
    for e in range(n_episodes):
        state = preprocess_state(env.reset())
        done = False
        i=0
        while not done:
            action = choose_action(state,get_epsilon(e))
            next_state,reward,done , _ = env.step(action)
            env.render()
            next_state = preprocess_state(next_state)
            remember(state,action,reward,next_state,done)
            state=next_state
            i=i+1
        scores.append(i)
        mean_score = np.mean(scores)
        
        if(mean_score >= n_win_ticks and e >100):
            if not quiet : print ("Ran {} episodes. Solved after {} trails".format(e,e-100))
            return e -100
        
        if e%100==0 and not quiet:
            print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks '.format(e,mean_score))
            
        replay(batch_size,get_epsilon(e))
        
    if not quiet: print('Did not solve after {} episodes'.format(e))
    return e



run()


Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\DELL\Anaconda2\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


[Episode 0] - Mean survival time over last 100 episodes was 13.0 ticks 
[Episode 100] - Mean survival time over last 100 episodes was 9.81 ticks 
[Episode 200] - Mean survival time over last 100 episodes was 9.4 ticks 
[Episode 300] - Mean survival time over last 100 episodes was 11.35 ticks 
[Episode 400] - Mean survival time over last 100 episodes was 24.46 ticks 
[Episode 500] - Mean survival time over last 100 episodes was 41.9 ticks 
[Episode 600] - Mean survival time over last 100 episodes was 42.96 ticks 
[Episode 700] - Mean survival time over last 100 episodes was 63.36 ticks 
